# 準備

今回の実習では、これまでに利用してきた jupyterlab と pandas に加えて、可視化ライブラリの Bokeh パッケージを利用します。あらかじめインストールしておいて下さい。

- 2019-12-19: 講義中に見つけた無駄な文を削除しました。
- 2019-12-19: 「カテゴリ変数に応じた色分け」の例題で、3つのチャートの pan 操作を連携するためのコードを追加しました。
- おまけ

    今日の授業の最初に紹介した某翻訳プロジェクトを[密かに見えるようにしています](https://wakita.github.io/y19-vis/i68wcp/vabook/toc/)。
    
    履修者以外には漏らさないで下さい。

In [13]:
# 基本モジュールの読み込み

import math

from IPython.core.display import display, HTML, Markdown

import numpy  as np
import pandas as pd

pd.options.display.max_rows = 8

In [14]:
# データの読み込み

SPI         = 'Social Progress Index'
NEEDS       = 'Basic Human Needs'
WELLBEING   = 'Foundations of Wellbeing'
OPPORTUNITY = 'Opportunity'

# ご存知の SPI のデータです。以前、配布した data/spi.xlsx は 2018 年版のデータでした。こちらは最新版です。
spi19 = pd.read_excel('../data/spi2019.xlsx', sheet_name=None)['2019'][['Country', 'Code', SPI, NEEDS, WELLBEING, OPPORTUNITY]]
# 国連が配布している国連に加盟する国や地域のデータです。国名前、各種コードなどが地域階層に整理されています。
world = pd.read_excel('../data/UNSD-M49.xlsx')

# ISO Code of Eastern Asia countries and regions
def subregion(region):
    code = list(world[world['Sub-region Name'] == region]['ISO-alpha3 Code'])
    return spi19.loc[spi19['Code'].isin(code)]

EAsia, SAsia, WAsia, NAsia, CAsia = [subregion(d + ' Asia')
                                     for d in ['Eastern', 'Southern', 'Western', 'Northern', 'Central']]

# spi と world のデータセットを ISO-α3 コードを用いて紐づけます。spi ではコードの名称は Code となっています。
spi19 = spi19.join(world[['ISO-alpha3 Code', 'Region Name', 'Sub-region Name']].set_index('ISO-alpha3 Code'), on='Code').dropna()
regions, subregions = set(spi19['Region Name']), set(spi19['Sub-region Name'])
display([regions, subregions])

[{'Africa', 'Americas', 'Asia', 'Europe', 'Oceania'},
 {'Australia and New Zealand',
  'Central Asia',
  'Eastern Asia',
  'Eastern Europe',
  'Latin America and the Caribbean',
  'Melanesia',
  'Northern Africa',
  'Northern America',
  'Northern Europe',
  'South-eastern Asia',
  'Southern Asia',
  'Southern Europe',
  'Sub-Saharan Africa',
  'Western Asia',
  'Western Europe'}]

In [15]:
# Bokeh のモジュールの読み込み

from bokeh.plotting     import *
from bokeh.models       import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.transform    import factor_cmap

In [16]:
# 散布図を作成（表示するのは後述の show()）
p = figure(title=f'{SPI} vs {OPPORTUNITY} in the world')
p.circle(x=SPI, y=OPPORTUNITY,
         source=ColumnDataSource(spi19),
         size=10)  # 点の大きさ
p.xaxis.axis_label=SPI;  p.yaxis.axis_label=OPPORTUNITY

# ホバーツールの追加
def add_hover(p):
    hover = HoverTool()
    # HoverTool の tooltips には (表示ラベル, @データの列名) を指定する。
    # ただし、列名が空白を含む場合は列名を { と } で囲う。
    # https://docs.bokeh.org/en/latest/docs/user_guide/tools.html
    hover.tooltips = [('Country', '@Country')] + [
        (i, '@{' + i + '}{0.0}')
        for i in ['Social Progress Index',
                  'Basic Human Needs',
                  'Foundations of Wellbeing',
                  'Opportunity']]
    p.add_tools(hover)

add_hover(p)

# 図の表示
output_notebook(); show(p)

Loading BokehJS ...

In [17]:
# ほかの3指標についても散布図を作成

def scatter_SPI(index):
    p = figure(title=f'{SPI} vs {index} in the world',
               x_range=(0, 100), y_range=(0, 100))
    p.circle(x=SPI, y=index,
             source=ColumnDataSource(spi19),
             size=10)  # 点の大きさ
    p.xaxis.axis_label=SPI;  p.yaxis.axis_label=index
    
    add_hover(p)

    # 図の表示
    output_notebook(); show(p)

for index in [NEEDS, WELLBEING, OPPORTUNITY]: scatter_SPI(index)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [18]:
# 3つの指標について併置

from bokeh.layouts import column, row

def scatter_SPI(index, spi):
    p = figure(title=f'{SPI} vs {index} in the world',
               plot_width=400, plot_height=400,
               x_range=(0, 100), y_range=(0, 100))
    p.circle(x=SPI, y=index, source=ColumnDataSource(spi),
             size=10)  # 点の大きさ
    p.xaxis.axis_label=SPI;  p.yaxis.axis_label=index

    add_hover(p)
    return p

    # 図の表示: output_notebook の前に reset_output しないと空のタブが表示される
    reset_output(); output_notebook(); show(p)

figures = [scatter_SPI(index, EAsia) for index in [NEEDS, WELLBEING, OPPORTUNITY]]
output_notebook(); show(row(figures))

Loading BokehJS ...

In [19]:
# アジアの5地域を重畳 (overlay)

p = figure(title=f'{SPI} vs {index} in the world', x_range=(0, 100), y_range=(0, 100))

def scatter_SPI(p, region, index, color):
    p.circle(x=SPI, y=index, source=ColumnDataSource(region), size=10, color=color)
    p.xaxis.axis_label=SPI;  p.yaxis.axis_label=index

for region, color in zip([EAsia, SAsia, WAsia, NAsia, CAsia], ['blue', 'red', 'green', 'cyan', 'pink']):
    scatter_SPI(p, region, OPPORTUNITY, color)
add_hover(p)
output_notebook(); show(p)

Loading BokehJS ...

In [20]:
# カテゴリ変数に応じた色分け

from bokeh.palettes import Category20
region_cmap = factor_cmap('Region Name', palette=Category20[len(regions)], factors=list(regions))

def scatter_SPI(spi, index):
    p = figure(title=f'{SPI} vs {index} in the world', x_range=(80, 95), y_range=(80, 95), plot_width=400, plot_height=400)
    p.circle(x=SPI, y=index, source=ColumnDataSource(spi), size=10, color=region_cmap,
            legend_field='Region Name')
    p.xaxis.axis_label=SPI;  p.yaxis.axis_label=index
    p.legend.location = "top_left"
    add_hover(p)
    return p

f0, f1, f2 = [scatter_SPI(spi19, index) for index in [NEEDS, WELLBEING, OPPORTUNITY]]
for f in [f1, f2]:
    f.x_range = f0.x_range; f.y_range = f0.y_range
output_notebook(); show(column(row(f0, f1), f2))

Loading BokehJS ...

In [21]:
EAsianCountries = list(EAsia['Country'])

# 棒グラフの作成
p = figure(title='Opportunity in East Asai',
           plot_width=800,
           x_range=EAsianCountries, y_range=(0, 100))
p.vbar(x='Country', top=OPPORTUNITY, source=ColumnDataSource(EAsia),
       width=0.9, legend_field='Country',
       fill_color=factor_cmap('Country', palette=Category20[len(EAsianCountries)], factors=EAsianCountries))
p.xaxis.axis_label='Country'
p.xaxis.major_label_orientation = math.pi/12
p.yaxis.axis_label=OPPORTUNITY

add_hover(p)

# 図の表示
output_notebook(); show(p)

Loading BokehJS ...

# 課題

2019年版の SPI データセットについての分析し、そこで見つけた面白い知見を Markdown 記述と静的可視化を組み合わせる形で効果的な静的インフォグラフィックスにまとめることがこの課題の目的です。

1. `labs/lab2/lab2.ipynb` を複製して `labs/lab2/mylab2.ipynb` と命名し、描画方法、表示する SPI の属性、Bokeh のパラメター調整について変更を施して Bokeh の使い方に慣れて下さい。さらに、この SPI データセットについての分析をし、知見をまとめなさい。**このファイルは提出しません。**

1. `labs/lab2/infographics.ipynb` という新しいノートブックを作成し、そこに文章と静的な画像を組み合わせたインフォグラフィックスを完成しなさい。

1. 下記の要領で Dropbox file request に `labs/lab2/infographics.ipynb` を提出しなさい。提出にあたって、Dropbox account は不要です。提出するファイルは `info-graphics.ipynb` のままで結構です。アップロードすると Dropbox から私に自動的にメールが配信されますので、わたしも確認できます。

- 〆切: 12月25日

- [提出先](https://www.dropbox.com/request/i5JHP4EUg5zhlHnGhlqr)

- 12月25日の実習 (VIS-LX5) では、各自持ち時間2分で提出されたインフォグラフィクスを用いてプレゼンをしていただきます。